In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
import torch.nn.functional as F
from matplotlib import pyplot as plt
import threading
# Загрузка модели PyTorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
state_dict = torch.load('C:/Users/work/FILTERS/Models/model_weave_1024_x_circ_512_ep14_12.pth')['model_state_dict']
model = UNet(3)
model.load_state_dict(state_dict, strict=True)
model.eval()  # Переводим модель в режим оценки

# Трансформации для входного изображения
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Функция для предобработки кадра
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = transform(frame)
    frame = frame.unsqueeze(0)  # Добавляем размерность батча
    return frame

# Функция для постобработки маски
def postprocess_mask(mask):
    
    
    mask[mask<0.76] = 0
    
    return mask

# Функция для наложения маски
def overlay_mask_on_frame(frame, mask, alpha=0.5):
    mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
    color_mask = np.zeros_like(frame)
    color_mask[mask == 255] = [0, 0, 255]  # Красный цвет для сегментации
    overlayed = cv2.addWeighted(frame, 1 - alpha, color_mask, alpha, 0)
    return overlayed

# Обработка видео
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    with torch.no_grad():  # Отключаем вычисление градиентов
        while cap.isOpened():
            start_time = time.time()
            ret, frame = cap.read()
            if not ret:
                break
                
            # Предобработка и преобразование в тензор PyTorch
            input_tensor = preprocess_frame(frame)
            
            # Передаем данные через модель
            if torch.cuda.is_available():
                input_tensor = input_tensor.cuda()
            
            output = model(input_tensor)
            output = F.softmax(output, dim=1)
            # Преобразуем выход модели в маску
            mask = output.squeeze().cpu().numpy()  # Удаляем размерности батча и каналов
            
            # Постобработка и наложение маски
            mask = postprocess_mask(mask[1])
            
            result_frame = overlay_mask_on_frame(frame, mask)
            plt.imshow(result_frame)
            # Сохранение и отображение
            
            out.write(result_frame)
            elapsed_time = time.time() - start_time  
            time.sleep(max(0, 1 - elapsed_time))
            
           
    
    cap.release()
    out.release()
    

# Пример использования
input_video = '4_3.mp4'
output_video = 'output_segmented.mp4'
process_video(input_video, output_video)